## Dependecies

In [1]:
# Dependencies and Setup
import pandas as pd
from sqlalchemy import create_engine

 ## Store CSV into DataFrame

In [2]:
# Read the SNIIV CSV

sniiv_file = "Resources/sniiv.csv"
sniiv_df = pd.read_csv(sniiv_file, encoding='latin-1')
sniiv_df.head()

,Año,Mes acumulado,Clave Entidad INEGI,Entidad Federativa,Clave Municipio INEGI,Municipio,Organismo,Modalidad,Destino del Credito,Tipo de Credito,Genero,Rango de Edad,Rango de Ingresos UMA,Valor de la Vivienda,Acciones,Monto
0,2020,1,17,Morelos,18,Temixco,BANCA (CNBV),Mejoramientos,Mejoramientos,Credito individual,Hombre,29 o menos,2.6 o menos,Económica,1,"17,209.00"
1,2020,1,4,Campeche,3,Carmen,BANCA (CNBV),Mejoramientos,Mejoramientos,Credito individual,Hombre,30 a 59,4.01 a 6.00,Económica,3,"158,963.00"
2,2020,1,9,Ciudad de México,2,Azcapotzalco,BANCA (CNBV),Mejoramientos,Mejoramientos,Credito individual,Hombre,29 o menos,2.6 o menos,Económica,1,"19,516.00"
3,2020,1,13,Hidalgo,51,Mineral de la Reforma,BANCA (CNBV),Mejoramientos,Mejoramientos,Credito individual,Hombre,30 a 59,4.01 a 6.00,Económica,1,"57,534.00"
4,2020,1,2,Baja California,4,Tijuana,BANCA (CNBV),Mejoramientos,Mejoramientos,Credito individual,Mujer,30 a 59,2.61 a 4.00,Económica,7,"340,331.00"


## SNIIV - Data Transforming

In [3]:
# Get a list of all of our columns for easy reference.

sniiv_df.columns

Index(['Año', 'Mes acumulado', 'Clave Entidad INEGI', 'Entidad Federativa',
       'Clave Municipio INEGI', 'Municipio', 'Organismo', 'Modalidad',
       'Destino del Credito', 'Tipo de Credito', 'Genero', 'Rango de Edad',
       'Rango de Ingresos UMA', 'Valor de la Vivienda', 'Acciones', 'Monto'],
      dtype='object')

In [4]:
# Reduce DF to allocate just the needed columns.

sniiv_analysis_df = sniiv_df.loc[:, ["Clave Municipio INEGI", "Genero", "Rango de Edad","Monto"]]
sniiv_analysis_df

,Clave Municipio INEGI,Genero,Rango de Edad,Monto
0,18,Hombre,29 o menos,"17,209.00"
1,3,Hombre,30 a 59,"158,963.00"
2,2,Hombre,29 o menos,"19,516.00"
3,51,Hombre,30 a 59,"57,534.00"
4,4,Mujer,30 a 59,"340,331.00"
...,...,...,...,...
355369,31,Hombre,30 a 59,"88,527.00"
355370,142,Mujer,30 a 59,"105,777.00"
355371,68,Hombre,29 o menos,"66,281.00"
355372,21,Hombre,30 a 59,"180,938.00"


In [5]:
# Replace the values of columns "Genero" & "Rango de Edad" with keys (https://datatofish.com/replace-values-pandas-dataframe/)

sniiv_analysis_df["Genero"] = sniiv_analysis_df["Genero"].replace(["Hombre","Mujer"],[0,1])

sniiv_analysis_df["Rango de Edad"] = sniiv_analysis_df["Rango de Edad"].replace(["29 o menos","30 a 59","60 o más"],[0,1,2])

sniiv_analysis_df

,Clave Municipio INEGI,Genero,Rango de Edad,Monto
0,18,0,0,"17,209.00"
1,3,0,1,"158,963.00"
2,2,0,0,"19,516.00"
3,51,0,1,"57,534.00"
4,4,1,1,"340,331.00"
...,...,...,...,...
355369,31,0,1,"88,527.00"
355370,142,1,1,"105,777.00"
355371,68,0,0,"66,281.00"
355372,21,0,1,"180,938.00"


## Cleaned SNIIV Table

In [6]:
# Rename the columns a create new data frame / Reset the index

sniiv_final_df = sniiv_analysis_df.rename(columns = {"Clave Municipio INEGI":"Clave_Municipio", "Genero":"ID_Genero", 
                                                                 "Rango de Edad":"ID_Rango_Edad","Monto":"Prestamo"}, inplace=False)

sniiv_final_df = sniiv_final_df.reset_index(drop=True)

sniiv_final_df

,Clave_Municipio,ID_Genero,ID_Rango_Edad,Prestamo
0,18,0,0,"17,209.00"
1,3,0,1,"158,963.00"
2,2,0,0,"19,516.00"
3,51,0,1,"57,534.00"
4,4,1,1,"340,331.00"
...,...,...,...,...
355369,31,0,1,"88,527.00"
355370,142,1,1,"105,777.00"
355371,68,0,0,"66,281.00"
355372,21,0,1,"180,938.00"


## Genero - Table

In [7]:
# Create a Table of "Genero" according to the parameters of SNIIV data set.

data = {'ID': [0, 1], 'Genero': ["Hombre", "Mujer"]}
gender_df = pd.DataFrame(data=data)
gender_df

,ID,Genero
0,0,Hombre
1,1,Mujer


## Rango Edad - Table

In [8]:
# Create a Table of "Genero" according to the parameters of SNIIV data set.

data = {'ID': [0, 1,2], 'Rango': ["29 o menos","30 a 59","60 o más"]}
rango_edad_df = pd.DataFrame(data=data)
rango_edad_df

,ID,Rango
0,0,29 o menos
1,1,30 a 59
2,3,60 o más
